# MCMC 기반 합성 의료 데이터 생성기

> **Purpose:** MCMC(Markov Chain Monte Carlo) 알고리즘을 활용하여 의학적 정합성을 갖춘 합성 환자 데이터를 생성합니다.
> - 국가 표준 의료 데이터(NHIS, HIRA) 메타데이터 기반 사전분포 정의
> - 혈압↔맥박 등 변수 간 임상적 상관관계 반영
> - 질병 발생률의 확률적 보정
>
> **출력 컬럼:** `patient_id`, `timestamp`, `heart_rate`, `systolic_bp`, `diastolic_bp`, `spo2`, `temperature`, `respiratory_rate`

## 1. 라이브러리 및 설정

In [ ]:

import numpy as np
import pandas as pd
from scipy import stats
from pyspark.sql import functions as F
from pyspark.sql.types import *

# 재현성을 위한 랜덤 시드
np.random.seed(42)

# 생성할 환자 수
NUM_PATIENTS = 10000
NUM_VITAL_RECORDS_PER_PATIENT = 50

print(f"🔧 설정: {NUM_PATIENTS}명 환자, 환자당 {NUM_VITAL_RECORDS_PER_PATIENT}건 바이탈 레코드")

## 2. 사전분포 정의 (Prior Distribution)

의료인 자문 및 의료통계 논문 기반으로 사전분포를 정의합니다.

In [ ]:

# 환자 인구통계학적 분포 (NHIS 표본 코호트 기반)
AGE_DISTRIBUTION = {
    "mean": 55, "std": 18, "min": 18, "max": 95
}

GENDER_RATIO = {"M": 0.48, "F": 0.52}

BMI_DISTRIBUTION = {
    "mean": 24.5, "std": 3.8, "min": 15.0, "max": 45.0
}

# 바이탈 사인 정상 범위 (생리학적 상관관계 포함)
# P2T2 실제 컬럼명: snake_case
VITAL_PRIORS = {
    "heart_rate":        {"mean": 75, "std": 12, "min": 40, "max": 180},
    "systolic_bp":       {"mean": 120, "std": 15, "min": 80, "max": 200},
    "diastolic_bp_ratio": {"mean": 0.65, "std": 0.05},  # diastolic ≈ systolic × 0.65
    "spo2":              {"mean": 97, "std": 1.5, "min": 85, "max": 100},
    "temperature":       {"mean": 36.8, "std": 0.4, "min": 35.0, "max": 41.0},
    "respiratory_rate":  {"mean": 16, "std": 3, "min": 8, "max": 40},
}

# 질병 발생률 (HIRA 보건의료빅데이터 참조)
DISEASE_PREVALENCE = {
    "hypertension": 0.28,
    "diabetes": 0.14,
    "cardiac_arrest": 0.02,
    "stroke": 0.03,
    "respiratory_failure": 0.05,
    "trauma": 0.15,
    "normal": 0.33,
}

print("✅ 사전분포 정의 완료")

## 3. MCMC 샘플링 (Metropolis-Hastings)

In [ ]:

def mcmc_vital_sampler(n_samples, prior, burn_in=1000):
    """
    Metropolis-Hastings MCMC 알고리즘을 사용하여
    임상적 상관관계가 반영된 바이탈 사인 데이터를 생성합니다.
    """
    total_samples = n_samples + burn_in
    
    # 초기값 설정
    current = {
        "heart_rate": prior["heart_rate"]["mean"],
        "systolic_bp": prior["systolic_bp"]["mean"],
        "spo2": prior["spo2"]["mean"],
        "temperature": prior["temperature"]["mean"],
        "respiratory_rate": prior["respiratory_rate"]["mean"],
    }
    
    samples = {key: [] for key in current.keys()}
    samples["diastolic_bp"] = []
    
    accepted = 0
    
    for i in range(total_samples):
        # 제안 분포에서 새로운 값 생성
        proposal = {}
        for key in current:
            p = prior[key]
            proposal[key] = current[key] + np.random.normal(0, p["std"] * 0.1)
            proposal[key] = np.clip(proposal[key], p["min"], p["max"])
        
        # 임상적 상관관계 반영 (혈압↔맥박 역상관)
        if proposal["systolic_bp"] < current["systolic_bp"]:
            proposal["heart_rate"] += np.random.uniform(2, 8)
        elif proposal["systolic_bp"] > current["systolic_bp"]:
            proposal["heart_rate"] -= np.random.uniform(1, 5)
        
        proposal["heart_rate"] = np.clip(
            proposal["heart_rate"], 
            prior["heart_rate"]["min"], 
            prior["heart_rate"]["max"]
        )
        
        # SpO2↔호흡수 상관관계
        if proposal["spo2"] < 94:
            proposal["respiratory_rate"] += np.random.uniform(2, 6)
        
        proposal["respiratory_rate"] = np.clip(
            proposal["respiratory_rate"],
            prior["respiratory_rate"]["min"],
            prior["respiratory_rate"]["max"]
        )
        
        # 수용 확률 계산 (log-likelihood 기반)
        log_alpha = 0
        for key in current:
            p = prior[key]
            log_alpha += (
                stats.norm.logpdf(proposal[key], p["mean"], p["std"])
                - stats.norm.logpdf(current[key], p["mean"], p["std"])
            )
        
        # 수용/거절
        if np.log(np.random.uniform()) < log_alpha:
            current = proposal.copy()
            accepted += 1
        
        # burn-in 이후 샘플 기록
        if i >= burn_in:
            for key in current:
                samples[key].append(round(current[key], 1))
            
            # diastolic_bp 생성 (systolic_bp와의 상관관계)
            dia_ratio = np.random.normal(
                prior["diastolic_bp_ratio"]["mean"], 
                prior["diastolic_bp_ratio"]["std"]
            )
            samples["diastolic_bp"].append(round(current["systolic_bp"] * dia_ratio))
    
    acceptance_rate = accepted / total_samples
    print(f"📊 MCMC 수용률: {acceptance_rate:.2%} (권장: 20%~50%)")
    
    return samples

## 4. 환자 프로필 및 바이탈 데이터 생성

In [ ]:

def generate_patient_profiles(n_patients):
    """환자 인구통계학적 프로필 생성"""
    patients = []
    for i in range(n_patients):
        patient_id = f"M{str(i+1).zfill(5)}"
        age = int(np.clip(
            np.random.normal(AGE_DISTRIBUTION["mean"], AGE_DISTRIBUTION["std"]),
            AGE_DISTRIBUTION["min"], AGE_DISTRIBUTION["max"]
        ))
        gender = np.random.choice(["M", "F"], p=[GENDER_RATIO["M"], GENDER_RATIO["F"]])
        bmi = round(np.clip(
            np.random.normal(BMI_DISTRIBUTION["mean"], BMI_DISTRIBUTION["std"]),
            BMI_DISTRIBUTION["min"], BMI_DISTRIBUTION["max"]
        ), 1)
        
        condition = np.random.choice(
            list(DISEASE_PREVALENCE.keys()),
            p=list(DISEASE_PREVALENCE.values())
        )
        
        patients.append({
            "patient_id": patient_id,
            "age": age,
            "gender": gender,
            "bmi": bmi,
            "condition": condition
        })
    
    return patients

patient_profiles = generate_patient_profiles(NUM_PATIENTS)
print(f"✅ {len(patient_profiles)}명 환자 프로필 생성 완료")

vital_samples = mcmc_vital_sampler(
    n_samples=NUM_PATIENTS * NUM_VITAL_RECORDS_PER_PATIENT,
    prior=VITAL_PRIORS
)

print(f"✅ {NUM_PATIENTS * NUM_VITAL_RECORDS_PER_PATIENT}건 바이탈 레코드 생성 완료")

## 5. Spark DataFrame 변환 및 저장

In [ ]:

spark.sql("USE CATALOG P2T2")

# 환자 프로필 DataFrame
df_patients = spark.createDataFrame(patient_profiles)

# 바이탈 DataFrame (P2T2 snake_case 컬럼명)
vital_records = []
for idx in range(len(vital_samples["heart_rate"])):
    patient_idx = idx // NUM_VITAL_RECORDS_PER_PATIENT
    if patient_idx < len(patient_profiles):
        vital_records.append({
            "patient_id": patient_profiles[patient_idx]["patient_id"],
            "timestamp": pd.Timestamp.now() - pd.Timedelta(minutes=idx),
            "heart_rate": int(vital_samples["heart_rate"][idx]),
            "systolic_bp": int(vital_samples["systolic_bp"][idx]),
            "diastolic_bp": int(vital_samples["diastolic_bp"][idx]),
            "spo2": int(vital_samples["spo2"][idx]),
            "temperature": float(vital_samples["temperature"][idx]),
            "respiratory_rate": int(vital_samples["respiratory_rate"][idx]),
        })

df_vitals = spark.createDataFrame(vital_records)

# Bronze Layer에 저장 (P2T2.bronze)
df_vitals.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("P2T2.bronze.vital_signs")

print("✅ P2T2.bronze.vital_signs 저장 완료")
print(f"   - vital_signs: {df_vitals.count()} 건")
print(f"   - Columns: {df_vitals.columns}")